In [2]:
import gc
import os
import random
import numpy as np
import pandas as pd

from matplotlib import pyplot as plt
import seaborn as sns

# Reading Data

In [27]:
# Reading Training Data
'''
- gsr_test.csv - Galvanic skin response sampled at 1Hz.
- hr_test.csv - Heart Rate sampled at 1Hz.
- rr_test.csv - RR intervals resampled at 1Hz.
- temp_test.csv - Skin temperature sampled at 1Hz.
- labels_train.csv - The first column contains the data labels (0 - cognitive load, 1-resting). 
The second column contains the user_id.

Each file contains 632 lines x 30 columns, 
corresponding to 632 instances each containing 
30 samples (30 seconds at the sampling rate 1 Hz). 
The train instances are randomly permuted.
'''
# no missing values

gsr_train = pd.read_csv('train/gsr_train.csv', header=None) 
hr_train = pd.read_csv('train/hr_train.csv', header=None)
rr_train = pd.read_csv('train/rr_train.csv',header=None)
labels_train = pd.read_csv('train/labels_train.csv', header=None)
temp_train = pd.read_csv('train/temp_train.csv',header=None)

In [16]:
gsr_train.head()

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,1.594915,1.596633,1.599173,1.601720,1.607820,1.612109,1.610408,1.599188,1.594050,1.595751,...,1.652902,1.651074,1.651983,1.654720,1.652905,1.650186,1.644740,1.647467,1.655705,1.666707
1,0.655883,0.656599,0.657755,0.658620,0.659052,0.658040,0.657175,0.656743,0.656743,0.656743,...,0.365144,0.369245,0.376029,0.384373,0.396744,0.492764,0.586921,0.673100,0.671894,0.669798
2,1.047488,1.048219,1.046757,1.045664,1.044205,1.043841,1.043479,1.042754,1.042029,1.039868,...,1.007824,0.998356,0.993391,0.989130,0.985875,0.982966,0.978498,0.965499,0.949937,0.938108
3,1.618148,1.601077,1.589129,1.574009,1.567478,1.559263,1.555212,1.552805,1.547205,1.538512,...,1.453495,1.451379,1.450678,1.449977,1.449977,1.452793,1.457031,1.464880,1.470614,1.474926
4,0.378287,0.380906,0.382021,0.382167,0.381875,0.381243,0.380952,0.380807,0.380325,0.379988,...,0.375658,0.376129,0.376601,0.376932,0.376743,0.376553,0.376364,0.376223,0.375799,0.374909


In [17]:
hr_train.head()

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,60.000000,60.666667,61.666667,62.000000,62.000000,62.000000,62.000000,62.000000,62.000000,62.000000,...,62.000000,61.666667,61.333333,61.333333,63.000000,64.333333,65.333333,65.333333,65.666667,66.000000
1,75.333333,75.666667,75.333333,75.000000,75.000000,75.000000,74.666667,74.000000,73.666667,73.666667,...,78.333333,77.666667,75.666667,73.666667,72.000000,72.000000,71.666667,71.666667,71.666667,71.666667
2,61.666667,62.333333,63.000000,63.000000,63.000000,63.000000,63.000000,63.000000,63.000000,63.000000,...,63.333333,63.333333,63.000000,63.000000,63.000000,62.333333,61.000000,59.666667,58.666667,58.333333
3,77.333333,76.000000,75.333333,75.000000,75.000000,75.000000,75.000000,75.000000,75.000000,75.333333,...,76.000000,76.000000,75.666667,75.666667,75.333333,75.333333,75.000000,74.000000,72.666667,71.666667
4,76.000000,76.666667,77.333333,77.666667,77.666667,77.333333,76.666667,76.666667,76.333333,76.000000,...,76.333333,76.333333,76.333333,76.000000,75.666667,67.666667,60.000000,52.000000,52.333333,52.666667


In [18]:
rr_train.head()

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,0.995520,0.984459,0.956805,0.934683,0.929152,0.945744,0.967867,0.978928,0.967867,0.956805,...,0.984459,0.989989,0.978928,0.962336,0.907029,0.851723,0.829600,0.857253,0.901499,0.929152
1,0.718987,0.768763,0.735579,0.779824,0.857253,0.940213,0.945744,0.851723,0.763232,0.741109,...,0.868315,0.818539,0.813008,0.818539,0.846192,0.857253,0.790885,0.785355,0.813008,0.851723
2,0.237819,0.199104,0.530944,0.956805,1.227808,1.106133,0.884907,0.829600,0.757701,0.829600,...,0.940213,0.923621,0.956805,1.012112,1.050827,1.111664,1.144848,1.172501,1.211216,1.117195
3,0.818539,0.818539,0.813008,0.807477,0.796416,0.785355,0.774293,0.763232,0.752171,0.752171,...,0.818539,0.829600,0.796416,0.807477,0.824069,0.846192,0.857253,0.840661,0.829600,0.801947
4,0.680272,0.790885,0.779824,0.785355,0.835131,0.779824,0.790885,0.741109,0.785355,0.757701,...,0.807477,0.790885,0.713456,0.575189,0.486699,0.326309,0.381616,0.387147,0.680272,0.796416


In [19]:
labels_train.head()

,0,1
0,0,5gpsc
1,1,wjxci
2,1,rc1in
3,0,ibvx8
4,1,yljm5


In [20]:
temp_train.head()

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,33.619999,33.619999,33.619999,33.619999,33.619999,33.619999,33.619999,33.619999,33.619999,33.619999,...,33.630001,33.630001,33.630001,33.630001,33.630001,33.630001,33.630001,33.630001,33.630001,33.630001
1,34.290001,34.290001,34.290001,34.290001,34.290001,34.290001,34.290001,34.290001,34.290001,34.290001,...,34.290001,34.290001,34.290001,34.290001,34.290001,34.290001,34.290001,34.290001,34.290001,34.290001
2,32.040001,32.040001,32.040001,32.040001,32.040001,32.040001,32.040001,32.040001,32.040001,32.046668,...,32.060001,32.060001,32.060001,32.060001,32.060001,32.060001,32.060001,32.060001,32.060001,32.060001
3,35.009998,35.009998,35.009998,35.009998,35.006666,35.003333,35.000000,35.000000,35.000000,35.000000,...,35.000000,35.000000,35.000000,35.000000,35.000000,35.000000,35.000000,35.000000,35.000000,35.000000
4,31.900000,31.900000,31.900000,31.900000,31.900000,31.900000,31.900000,31.900000,31.900000,31.900000,...,31.900000,31.900000,31.900000,31.900000,31.900000,31.900000,31.900000,31.900000,31.900000,31.900000


In [30]:
# Reading Testing Data
'''
Files:
- gsr_test.csv - Galvanic skin response sampled at 1Hz.
- hr_test.csv - Heart Rate sampled at 1Hz.
- rr_test.csv - RR intervals resampled at 1Hz.
- temp_test.csv - Skin temperature sampled at 1Hz.
- exampleTeam_predictions.csv - first column contains example predictions and it should be modified by the participants. The second column contains the user_id.

Each file contains 193 lines x 30 columns, 
corresponding to 193 instances each containing 
30 samples (30 seconds at the sampling rate 1 Hz). 
The test instances are randomly permuted.
'''
# No missing values
gsr_test = pd.read_csv('test/gsr_test.csv', header=None)
hr_test = pd.read_csv('test/hr_test.csv', header=None)
rr_test = pd.read_csv('test/rr_test.csv',header=None)
temp_test = pd.read_csv('test/temp_test.csv',header=None)
example_pred = pd.read_excel('test/exampleTeam_predictions.xlsx', header=None)


In [31]:
example_pred.head()

,0,1
0,0,bd47a
1,0,6frz4
2,0,iz3x1
3,0,3caqi
4,0,iz3x1


In [32]:
temp_test.head()

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,32.529999,32.529999,32.529999,32.529999,32.529999,32.529999,32.529999,32.549999,32.570000,32.590000,...,32.590000,32.590000,32.590000,32.590000,32.590000,32.590000,32.590000,32.590000,32.590000,32.590000
1,33.950001,33.950001,33.950001,33.950001,33.950001,33.950001,33.950001,33.950001,33.950001,33.950001,...,33.950001,33.950001,33.950001,33.950001,33.950001,33.950001,33.940000,33.929999,33.919998,33.919998
2,34.180000,34.180000,34.180000,34.180000,34.180000,34.180000,34.180000,34.180000,34.180000,34.180000,...,34.200001,34.200001,34.200001,34.200001,34.200001,34.200001,34.200001,34.200001,34.200001,34.200001
3,35.160000,35.160000,35.160000,35.160000,35.160000,35.160000,35.160000,35.160000,35.160000,35.160000,...,35.160000,35.160000,35.160000,35.160000,35.160000,35.160000,35.160000,35.160000,35.160000,35.160000
4,34.279999,34.279999,34.279999,34.279999,34.279999,34.279999,34.279999,34.279999,34.279999,34.279999,...,34.279999,34.279999,34.279999,34.279999,34.279999,34.279999,34.279999,34.279999,34.279999,34.279999


In [33]:
rr_test.head()

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,1.100603,1.084011,1.039765,1.034235,1.034235,1.067419,1.050827,1.045296,1.050827,1.034235,...,1.012112,1.023173,1.028704,1.072949,1.084011,1.095072,1.045296,1.039765,1.061888,1.067419
1,0.824069,0.813008,0.801947,0.575189,0.425861,0.293125,0.259941,0.331840,0.475637,0.663680,...,0.724517,0.707925,0.702395,0.702395,0.735579,0.702395,0.503291,0.536475,0.481168,0.707925
2,0.824069,0.829600,0.868315,0.790885,0.796416,0.790885,0.813008,0.768763,0.735579,0.702395,...,0.824069,0.807477,0.818539,0.801947,0.818539,0.801947,0.793651,0.754936,0.749405,0.752171
3,0.967867,1.012112,1.067419,1.089541,0.956805,0.984459,0.907029,0.862784,0.741109,0.746640,...,0.907029,0.868315,0.846192,0.835131,0.851723,0.813008,0.763232,0.735579,0.774293,0.608373
4,0.890437,0.907029,0.973397,1.039765,1.045296,0.940213,0.829600,0.696864,0.663680,0.696864,...,1.006581,0.962336,0.890437,0.862784,0.907029,0.907029,0.956805,0.912560,0.840661,0.724517


In [34]:
hr_test.head()

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,63.666667,63.000000,62.333333,61.666667,61.333333,60.666667,60.333333,59.666667,61.000000,62.000000,...,65.333333,66.666667,66.333333,65.000000,64.000000,62.666667,62.333333,64.333333,66.333333,68.000000
1,83.000000,82.333333,81.666667,81.333333,82.000000,82.666667,83.000000,83.333333,84.000000,84.333333,...,82.000000,82.000000,82.000000,82.000000,82.000000,82.000000,82.333333,82.666667,83.000000,83.000000
2,68.666667,69.333333,69.666667,71.666667,73.666667,75.000000,74.666667,73.666667,75.000000,76.333333,...,70.666667,71.000000,71.666667,72.333333,72.000000,72.000000,72.000000,72.000000,72.000000,74.666667
3,59.666667,60.000000,59.666667,59.333333,59.000000,59.000000,58.666667,58.666667,59.000000,59.666667,...,64.333333,67.000000,68.666667,69.666667,68.333333,69.333333,71.333333,73.666667,74.333333,74.333333
4,64.333333,64.666667,64.666667,65.000000,64.666667,66.666667,68.666667,71.333333,71.666667,72.000000,...,68.000000,67.666667,67.333333,67.666667,68.333333,68.666667,68.333333,68.000000,67.666667,67.666667


In [36]:
gsr_test.head()

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,6.451613,6.437827,6.424042,6.410256,6.410256,6.396647,6.383037,6.369427,6.369427,6.369427,...,6.369599,6.383037,6.369599,6.342552,6.315845,6.302577,6.289308,6.302577,6.302577,6.302577
1,1.708436,1.715360,1.744004,1.762999,1.780537,1.783777,1.796443,1.798577,1.794264,1.791046,...,1.719226,1.725131,1.723156,1.720198,1.704717,1.691263,1.704304,1.726830,1.734439,1.711758
2,2.433157,2.415609,2.397199,2.380068,2.368783,2.358499,2.360367,2.367854,2.367854,2.358605,...,2.257344,2.252260,2.248878,2.243839,2.235517,2.225551,2.218937,2.217295,2.214032,2.207534
3,2.795904,2.798512,2.803743,2.811656,2.816916,2.816916,2.811626,2.808989,2.808989,2.808989,...,2.801120,2.801120,2.798512,2.795904,2.795904,2.801135,2.803743,2.803743,2.801120,2.803743
4,2.992486,3.243803,3.484433,3.476506,3.433042,3.379021,3.333656,3.307640,3.300330,3.303973,...,3.147259,3.090337,3.036816,3.012176,3.006054,2.994244,2.970760,2.938369,2.915567,2.901441
